In [95]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from dateutil.parser import parse

In [108]:
df = pd.read_csv('F1_4502\LapAndWeather\LapWeather_Australian Grand Prix')
df = df.drop(['Time','DriverNumber', 'Unnamed: 0', 'LapNumber', 'PitOutTime', 'PitInTime', 'Sector1Time','Sector2Time','Sector3Time','Sector1SessionTime','Sector2SessionTime','Sector3SessionTime','SpeedI1','SpeedI2','SpeedST','IsPersonalBest','FreshTyre','Stint','SpeedFL','LapStartTime','Team','Driver','TrackStatus','IsAccurate'], axis=1)
df = df.dropna()
dfOutput = df['LapTime']
dfInput = df.drop(['LapTime'], axis=1)
dfInput = dfInput.replace({'ULTRASOFT': 0, 'SUPERSOFT': 1, 'SOFT': 2,})
train_inputs = dfInput.to_numpy()
train_targets = dfOutput.to_numpy()
for i in range(len(train_targets)):
    train_targets[i] = train_targets[i].replace('0 days ', '')
actual_train_targets = []
for time in train_targets:
    td = parse(time) - parse('00:00:00')
    seconds = td.total_seconds()
    actual_train_targets.append(seconds)
train_inputs = train_inputs.astype('float64')
print(train_inputs[0])
train_targets = np.array(actual_train_targets)

[  0.    4.   24.2  36.3 996.9   0.   38.2 296.    3.8]


In [116]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

class f1Dataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      # Apply scaling if necessary
      if scale_data:
          X = StandardScaler().fit_transform(X)
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]
      

class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(9, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

  
if __name__ == '__main__':
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Load Boston dataset
  
  # Prepare Boston dataset
  dataset = f1Dataset(train_inputs, train_targets)
  trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)
  
  # Initialize the MLP
  mlp = MLP()
  
  # Define the loss function and optimizer
  loss_function = nn.L1Loss()
  optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)
  
  # Run the training loop
  for epoch in range(0, 5): # 5 epochs at maximum
    
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
      inputs, targets = data
      inputs, targets = inputs.float(), targets.float()
      targets = targets.reshape((targets.shape[0], 1))
      
      # Zero the gradients
      optimizer.zero_grad()
      
      # Perform forward pass
      outputs = mlp(inputs)
      
      # Compute loss
      loss = loss_function(outputs, targets)
      
      # Perform backward pass
      loss.backward()
      
      # Perform optimization
      optimizer.step()
      
      # Print statistics
      current_loss += loss.item()
      if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

  # Process is complete.
  print('Training process has finished.')

Starting epoch 1
Loss after mini-batch     1: 0.192
Loss after mini-batch    11: 1.865
Loss after mini-batch    21: 1.868
Loss after mini-batch    31: 1.860
Loss after mini-batch    41: 1.817
Loss after mini-batch    51: 1.868
Loss after mini-batch    61: 1.826
Loss after mini-batch    71: 1.837
Loss after mini-batch    81: 1.855
Loss after mini-batch    91: 1.861
Starting epoch 2
Loss after mini-batch     1: 0.177
Loss after mini-batch    11: 1.854
Loss after mini-batch    21: 1.849
Loss after mini-batch    31: 1.852
Loss after mini-batch    41: 1.846
Loss after mini-batch    51: 1.859
Loss after mini-batch    61: 1.843
Loss after mini-batch    71: 1.826
Loss after mini-batch    81: 1.851
Loss after mini-batch    91: 1.839
Starting epoch 3
Loss after mini-batch     1: 0.199
Loss after mini-batch    11: 1.875
Loss after mini-batch    21: 1.809
Loss after mini-batch    31: 1.842
Loss after mini-batch    41: 1.845
Loss after mini-batch    51: 1.803
Loss after mini-batch    61: 1.819
Loss

In [112]:
#############################
#NOT BEING USED RIGHT NOW #######
##########################

# # Define the model
# class NeuralNet(nn.Module):
#     def __init__(self, input_size, hidden_size):
#         super(NeuralNet, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, 1)
    
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out

# # Create the model with input size and hidden layer size
# input_size = 9
# hidden_size = 16
# model = NeuralNet(input_size, hidden_size)

# # Define the loss function and optimizer
# criterion = nn.MSELoss()
# learning_rate = 0.01
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Train the model
# num_epochs = 10
# for epoch in range(num_epochs):
#     # Convert input and target data to PyTorch tensors
#     inputs = torch.from_numpy(train_inputs)
#     targets = torch.from_numpy(train_targets)

#     # Forward pass
#     outputs = model(inputs)
#     loss = criterion(outputs, targets)

#     # Backward and optimize
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     if (epoch+1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # # Test the model
# # model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
# # with torch.no_grad():
# #     inputs = torch.from_numpy(test_inputs)
# #     targets = torch.from_numpy(test_targets)
# #     outputs = model(inputs)
# #     loss = criterion(outputs, targets)
# #     print(f'Test Loss: {loss.item():.4f}')

<class 'torch.Tensor'>


RuntimeError: expected scalar type Float but found Double